In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tensorflow.keras.datasets.mnist import load_data
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [14]:
mnist = fetch_openml('mnist_784')

In [15]:
x_data = mnist['data'].values/255
x_data = x_data.reshape(-1,1,28,28)
y_data = mnist['target'].astype(np.int32)
y_data = np.eye(10)[y_data]

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.3,stratify=y_data)

x_train.shape,x_test.shape,y_train.shape,y_test.shape

((49000, 1, 28, 28), (21000, 1, 28, 28), (49000, 10), (21000, 10))

In [27]:
x= torch.FloatTensor( x_train)
y = torch.FloatTensor( y_train)

In [28]:
class CNN2(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=3,kernel_size=5, stride=1) # 5 by 5 3 채널
        self.conv2 = nn.Conv2d(in_channels=3,out_channels=10,kernel_size=5, stride=1)
        self.max1 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(10*10*10,50) # 1. (nbyn-kernel_size)/stride + 1
        self.fc2 = nn.Linear(50,10)
        self.sm = nn.Softmax(dim=1)

    def forward(self,x):
        #print('연산 전',x.shape)
        x = F.relu(self.conv1(x))
        #print('conv1 연산 후',x.shape)
        x = F.relu(self.conv2(x))
        #print('conv2 연산 후',x.shape)
        x = self.max1(x)
        #print('max 연산 후',x.shape)
        x = x.view(-1,10*10*10)
        #print('reshape 후',x.shape)
        x = F.relu(self.fc1(x)) # matmul(x,w)
        #print('fc1 연산 후',x.shape)
        x = self.fc2(x)
        #print('fc2 연산 후',x.shape)
        x = self.sm(x)
        return x

In [30]:
cnn = CNN2()

loss_fn = torch.nn.CrossEntropyLoss()

optimizer = Adam(cnn.parameters(),lr=0.01)

for epoch in range(500):
    optimizer.zero_grad()
    hx = cnn(x)
    c = loss_fn(hx,y)
    c.backward()
    optimizer.step()
    rdata = y.argmax(dim=1).numpy()
    pred_train = hx.argmax(dim=1).numpy()
    acc= accuracy_score(rdata,pred_train)
    print(epoch, c.item(), acc)

0 2.3028271198272705 0.09928571428571428
1 2.298853874206543 0.10202040816326531
2 2.2796268463134766 0.10202040816326531
3 2.236825466156006 0.20820408163265305
4 2.154456377029419 0.3243469387755102
5 2.050454616546631 0.4606122448979592
6 1.9472533464431763 0.5773265306122449
7 1.916522741317749 0.5568775510204081
8 1.8331900835037231 0.6567142857142857
9 1.7962677478790283 0.6768367346938775
10 1.7706063985824585 0.7055918367346938
11 1.737690806388855 0.7314897959183674
12 1.7198058366775513 0.7468163265306123
13 1.7006568908691406 0.764734693877551
14 1.6918516159057617 0.7759591836734694
15 1.6889592409133911 0.7777142857142857
16 1.6757084131240845 0.7900408163265306
17 1.6610807180404663 0.8026938775510204
18 1.653936505317688 0.8086122448979591
19 1.6535037755966187 0.8081836734693878
20 1.6515661478042603 0.8106122448979591
21 1.6459797620773315 0.8160816326530612
22 1.6402229070663452 0.8213061224489796
23 1.6367617845535278 0.824530612244898
24 1.6346464157104492 0.8272448